In [12]:
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Reusable classes
import sys
sys.path.append("/home/jovyan/work/upload/ml-experiments/sc_wrangling")

# Data path
import pathlib
FOLDER = pathlib.Path("/home/jovyan/work/upload/")
TABLE_PATH = FOLDER / 'data/pickle/df_columns_labels.pkl'
LAW2VEC_PATH = FOLDER / 'data/law2vec/Law2Vec.200d.txt'

# Manage ML lifecycle
import mlflow
from mlflow import log_params, set_tags, log_metrics
MLFLOW_TRACKING_URI = 'http://srv.meaningfy.ws:8989'

# Math stuff
import numpy as np
from numpy.core.records import ndarray

# Data visualisation
import pandas as pd
from pandas import Series

# Vectorize algotrithm
from gensim.models import Word2Vec, KeyedVectors

# Support Vector Classifier Algorithm
from sklearn.svm import SVC

# Intermediate steps of the pipeline must be ‘transforms’,
# that is, they must implement fit and transform methods.
from sklearn.pipeline import Pipeline

# Splitting data into train and test
from sklearn.model_selection import train_test_split

# Metrics Evaluation Methods
from evaluation_metrics import model_evaluation_metrics

In [2]:
# Read data
df = pd.read_pickle(TABLE_PATH)
# Independent data
columns = df['Concatenated Data (clean)']
# Label data
category = df['Subcategory']

## law2vec model

In [3]:
# Load a word2vec model stored in the C *text* format.
model = KeyedVectors.load_word2vec_format(LAW2VEC_PATH, binary=False)
l2v_dict = {w: vec for w, vec in zip(model.wv.index2word, model.wv.syn0)}


In [4]:
class MeanEmbeddingVectorizer(object):
    """Calculate the mean of each word"""
    def __init__(self, word2vec):
        self.word2vec = word2vec
        if len(word2vec)>0:
            self.dim=len(word2vec[next(iter(l2v_dict))])
        else:
            self.dim=0

    def fit(self, X, y):
        return self

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0) for words in X]
        )

In [51]:
# Use pipes to implement steps of fit and transform method
svm_l2v = Pipeline([
    # Add the words we want to mean
    ("law2vec vectorizer", MeanEmbeddingVectorizer(l2v_dict)),
    # Use SVC algorithm
    # :gamma: is a parameter for non linear hyperplanes. The higher the gamma
    #         value it tries to exactly fit the training data set.
    #
    # :C: is the penalty parameter of the error term. It controls the trade
    #     off between smooth decision boundary and classifying the training points correctly.
    #     !!! Increasing C values may lead to overfitting the training data. !!!
    ("SVM", SVC(gamma=3, C=2))])

## Train SVM Model

In [6]:
# split data into test and train sets
# where "train" contains 70% of data and "test" - 30%
X_train, X_test, y_train, y_test = train_test_split(
    columns, category, random_state=42, test_size=0.3, shuffle=True)

In [52]:
# Fit SVM model with out train data
svm_l2v.fit(X_train, y_train)

Pipeline(steps=[('law2vec vectorizer',
                 <__main__.MeanEmbeddingVectorizer object at 0x7f209f191670>),
                ('SVM', SVC(C=2, gamma=3))])

In [53]:
# Model Evaluation
svm_l2v.score(X_train, y_train)

0.9862385321100917

In [54]:
# SVM prediction based on test data
prediction = svm_l2v.predict(X_test)
prediction

array([14, 11,  0,  2, 20,  9, 10, 10, 10, 12, 10, 22, 10,  0,  0, 21, 12,
       33, 21,  9, 14, 10,  0, 10, 12,  0, 10,  2, 12, 20, 24, 14, 14, 10,
        0, 20, 21, 16,  2,  9,  4, 24, 26, 21, 14, 29, 30, 20,  9, 30, 26,
       20, 14, 26, 28, 14,  9, 14, 12,  9, 14, 10,  0,  2,  9,  0, 16, 14,
       20, 10,  2, 10,  0,  0, 27, 10, 12, 14, 15, 21, 24,  4,  0, 10, 10,
        0,  0, 21, 21, 10, 10,  0,  0, 14, 27, 13,  9, 10,  0, 10, 10, 10,
       14, 21, 14, 10, 10, 20,  9, 21,  2,  0,  9,  0, 22, 14,  2, 20,  9,
       10,  9, 14,  9, 12,  2, 26, 14, 26, 10, 20, 10,  9,  9,  9, 10, 21,
       10, 10,  2,  9, 19, 21,  4, 20, 21, 16, 10, 10,  2, 30,  0, 14, 18,
       14, 22,  2, 10,  0,  4, 10,  0, 21,  4, 14, 22, 10, 30, 10, 12,  9,
        2, 27, 30, 22, 20, 10, 10, 10, 13,  0, 26,  4, 14, 10,  0,  0,  0,
       10, 13, 10, 10, 21, 15, 10, 19, 30, 22,  0, 18, 30, 21, 22,  0, 14,
       10, 14,  4, 12,  0, 19, 14, 10, 10, 30,  0,  9,  2,  9, 10,  0, 10,
       16, 20, 27, 30, 10

## Evaluation

In [55]:
model_evaluation_metrics(y_test, prediction, 'Subategory')

,Evaluation Metrics,Subategory
0,Accuracy,0.555160
1,Precission,0.416806
2,Recall,0.376620
3,F1 Score,0.365535
4,Mean Absolute Error,4.896797
5,Mean Squared Error,82.804270


In [56]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment(experiment_name="Subategory (Law2Vec)")

with mlflow.start_run():

    parameters = {"Language model": 'law2vec',
                  "Random state": 42,
                  "Test size": 0.3,
                  "Gamma": 3,
                  "C": 2}
    log_params(parameters)
    metrics = { "Accuracy": 0.555160,
            "Precision": 0.416806,
            "Recall": 0.376620,
            "F1 Score": 0.365535,
            "Mean Absolute Error": 4.896797,
            "Mean Squared Error": 82.804270}
    log_metrics(metrics)